### Authored by Omar Hoang

In [ ]:
!pip install deepeval nest-asyncio

In [ ]:
import os
import pandas as pd
import nest_asyncio

from deepeval.metrics import GEval
from deepeval.test_case import LLMTestCase, LLMTestCaseParams
from google.colab import drive

# Set your OpenAI API key as an environment variable
os.environ['OPENAI_API_KEY'] = 'sk-proj-hh3M4kqA4aVuwWmsZ3c8T3BlbkFJsd3QlSGuDGr61WzRlRHi'
drive.mount('/content/drive')
nest_asyncio.apply()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/CS4371/GOT Results/GEval Results/Run 5/run5.csv') #path to questions
df = df.drop_duplicates(subset=['question'])

# Function to extract the part after "Helpful Answer: " and before the next newline
def extract_helpful_answer(answer):
    prefix = "Helpful Answer: "
    if prefix in answer:
        start_index = answer.find(prefix) + len(prefix)
        end_index = answer.find('\n', start_index)
        if end_index == -1:  # If no newline is found, take the rest of the string
            return answer[start_index:].strip()
        return answer[start_index:end_index].strip()
    return answer

# Define the marker line
start_marker = "Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer."
end_marker = "Question:"

# Function to extract the context lines between the markers
def extract_context_between_markers(answer):
    # Check if both markers are in the answer
    if start_marker in answer and end_marker in answer:
        # Find the indices of the markers
        start_index = answer.find(start_marker) + len(start_marker)
        end_index = answer.find(end_marker)
        # Get the text between the markers
        context_text = answer[start_index:end_index].strip()
        # Split the text into lines and filter out empty lines
        context_lines = [line.strip() for line in context_text.split('\n') if line.strip()]
        return context_lines
    return []

df['parsed_answer'] = df['answer'].apply(extract_helpful_answer)
df['context'] = df['answer'].apply(extract_context_between_markers)

In [ ]:
ground_truth = [
    "Ramsay Bolton is a sadistic character from House Bolton.",
    "House Stark rules the Kingdom of the North.",
    "Tyrion Lannister is the youngest sibling.",
    "Eddard Stark's sword is Ice.",
    "Jaime Lannister pushes Bran Stark.",
    "Members of House Stark include Eddard, Catelyn, Robb, Sansa, Arya, Bran, Rickon.",
    "Eddard Stark's appointment as Hand of the King increases Northern influence.",
    "Jaime Lannister is known as the Kingslayer.",
    "Daenerys Targaryen is the Mother of Dragons.",
    "Riverrun is the ancestral home of House Tully.",
    "Aerys II Targaryen is the father of Daenerys Targaryen.",
    "The continent where most of the story takes place is Westeros.",
    "Jon Snow's direwolf is named Ghost.",
    "Olenna Tyrell is known as the 'Queen of Thorns'.",
    "The Wall is made of ice.",
    "Mord is the Captain of the ship that takes Tyrion to King's Landing.",
    "Sandor Clegane is the Hound.",
    "Jon Snow’s sword is Longclaw.",
    "Varys's role impacts the power dynamics in King's Landing through intelligence and manipulation.",
    "Eddard Stark is the most honorable in the story.",
    "The motto of House Stark is 'Winter is Coming'.",
    "Daenerys Targaryen marries Khal Drogo.",
    "Samwell Tarly is the main steward of the Night's Watch and a close friend to Jon Snow.",
    "Arya Stark adopts a wolf pup named Nymeria.",
    "Rickon Stark is the youngest Stark son.",
    "The relationship between Theon Greyjoy and Ramsay Bolton affects Theon's character development deeply.",
    "Petyr Baelish uses his intelligence and manipulation to climb the political ladder.",
    "The first character to discover the existence of White Walkers in the prologue is Will.",
    "Petyr Baelish's full name is Petyr Baelish.",
    "The names of Daenerys’s three dragons are Drogon, Rhaegal, and Viserion.",
    "The Dothraki are referred to as horselords.",
    "House Tully rules The Riverlands.",
    "Winterfell is significant as the ancestral home of House Stark.",
    "Daenerys Targaryen's decisions have significant consequences in her quest to reclaim the Iron Throne.",
    "Jeor Mormont mentors Jon Snow at the Wall.",
    "Rickon Stark is the youngest child of Eddard and Catelyn Stark.",
    "Tyrion Lannister's marriage to Sansa Stark has complex political implications.",
    "Eddard Stark is appointed as Hand of the King after Jon Arryn's death.",
    "The maesters serve as scholars, healers, and advisors.",
    "Essos is the continent where the Dothraki live.",
    "The power struggle between the houses affects the stability of the Seven Kingdoms greatly.",
    "Lysa Arryn is the ruler of the Vale of Arryn.",
    "Khal Drogo is a Dothraki warlord and husband to Daenerys Targaryen.",
    "Robert Baratheon is the King of the Seven Kingdoms when the story begins.",
    "Tyrion Lannister's trial by combat has significant political ramifications.",
    "Shae is the name of Tyrion Lannister's lover from Essos.",
    "Arya Stark is trained by Jaqen H'ghar in Braavos.",
    "Jaime Lannister's character development throughout the series is significant.",
    "Dragons are mentioned frequently.",
    "The Iron Throne is the seat of the ruler of the Seven Kingdoms.",
    "Cersei Lannister is the mother of Joffrey, Myrcella, and Tommen.",
    "Ramsay Bolton is considered one of the evilest characters in the story.",
    "The events at the Red Wedding significantly change political alliances in Westeros."
]

# Add the list as a new column in the dataframe
df['ground_truth'] = ground_truth

In [ ]:
print(df['question'])

0                                 Who is Ramsay Bolton?
1            What House rules the Kingdom of the North?
2                Who is the youngest Lannister sibling?
3     What is the name of the Valyrian steel sword o...
4          Who pushes Bran Stark from the tower window?
5                   Who are the members of House Stark?
6     What are the political implications of Eddard ...
7                       Who is known as the Kingslayer?
8     Which character is known as the Mother of Drag...
9     What is the name of the ancestral home of Hous...
10             Who is the father of Daenerys Targaryen?
11    What is the name of the continent where most o...
12             What is the name of Jon Snow's direwolf?
13               Who is known as the 'Queen of Thorns'?
14            What is the name of the Wall made of ice?
15    Who is the Captain of the ship that takes Tyri...
16                                    Who is the Hound?
17                What is the name of Jon Snow’s

In [ ]:
print(df['context'])

0     [Harrion Karstark, four Freys. Lord Hornwood i...
1     [sworn allegiance to no man, styling themselve...
2     [— his widow, DARLESSA, of House Marbrand,, — ...
3     [blade? Tobho had learned to work Valyrian ste...
4     [gate, with a hundred feet of wall looming ove...
5     [Starks.”, — WYMAN MANDERLY, Lord of White Har...
6     [Eddard Stark, the Hand of the King,” he annou...
7     [“The Kingsguard—”, “The Kingslayer is massing...
8     [she grew old … and in her womb grew a son who...
9     [The principal houses sworn to the Eyrie are R...
10    [The Old Dynasty HOUSE TARGARYEN, His Name, Lo...
11    [children of the forest. Yet here and there in...
12    [their nights came alive with the howls of dis...
13    [called the Queen of Thorns,, — his sisters: —...
14    [brothers spread crushed stone across the walk...
15    [abducted Tyrion Lannister, thanks to Littlefi...
16    [at the Hound’s door, him and the cruel woman ...
17    [sometimes named a “bastard sword.” Yet th

In [ ]:
print(df['parsed_answer'])

0     Based solely upon these quotes provided by Geo...
1                Based on what we have learned so fa...
2     You seem unsure about who the Younger Brother ...
3     Ice! That would be correct because we are told...
4     The person who pushed Branthey doesnot existin...
5     I can provide information on who is part of ho...
6     Based solely upon what information provided wi...
7     Based off what was provided in text passage ab...
8     Ah! I can definitely help here 😊 It seems like...
9     Ah! Well... According tp this passage (emphasi...
10    Based off information provided within text pas...
11    Based on what we have been told thusfar...it s...
12    Based on information provided earlier (and wit...
13    Oh goodness gracious me! *adjust glasses* Why ...
14    I can see how this passage describes something...
15    Based off what information provided above...th...
16    Oh wow! That would require some actual knowled...
17    Based solely upon information provided wit

In [ ]:
print(df['ground_truth'])

0     Ramsay Bolton is a sadistic character from Hou...
1           House Stark rules the Kingdom of the North.
2             Tyrion Lannister is the youngest sibling.
3                          Eddard Stark's sword is Ice.
4                    Jaime Lannister pushes Bran Stark.
5     Members of House Stark include Eddard, Catelyn...
6     Eddard Stark's appointment as Hand of the King...
7           Jaime Lannister is known as the Kingslayer.
8          Daenerys Targaryen is the Mother of Dragons.
9        Riverrun is the ancestral home of House Tully.
10    Aerys II Targaryen is the father of Daenerys T...
11    The continent where most of the story takes pl...
12                  Jon Snow's direwolf is named Ghost.
13     Olenna Tyrell is known as the 'Queen of Thorns'.
14                             The Wall is made of ice.
15    Mord is the Captain of the ship that takes Tyr...
16                         Sandor Clegane is the Hound.
17                        Jon Snow’s sword is Lo

In [ ]:
# --EXAMPLE FROM DOCUMENTATION--

#correctness_metric = GEval(
#    name="Correctness",
#    criteria="Determine whether the actual output is factually correct based on the expected output.",
    # NOTE: you can only provide either criteria or evaluation_steps, and not both
#    evaluation_steps=[
#        "Check whether the facts in 'actual output' contradicts any facts in 'expected output'",
#        "You should also heavily penalize omission of detail",
#        "Vague language, or contradicting OPINIONS, are OK"
#    ],
#    evaluation_params=[LLMTestCaseParams.INPUT, LLMTestCaseParams.ACTUAL_OUTPUT],
#)

In [ ]:
# Generation-related metrics

# USE QUESTION AND ANSWER
correctness_metric = GEval(
    name="Correctness",
    criteria="Determine whether the actual output is factually correct based on the expected output.",
    evaluation_params=[LLMTestCaseParams.INPUT, LLMTestCaseParams.ACTUAL_OUTPUT],
)

# USE QUESTION (OR CONTEXT) AND ANSWER
answer_relevancy_metric = GEval(
    name="Answer Relevancy",
    criteria="Assess the relevancy of the answer in relation to the question.",
    evaluation_params=[LLMTestCaseParams.INPUT, LLMTestCaseParams.ACTUAL_OUTPUT],
)

In [ ]:
# Iterate through the dataframe and evaluate each row
correctness_results = []
answer_relevancy_results = []

for index, row in df.iterrows():

    test_case = LLMTestCase(
        input=row['question'],
        actual_output=row['parsed_answer'],
        expected_output=row['ground_truth'],
        context=row['context']
    )

    correctness_metric.measure(test_case)
    answer_relevancy_metric.measure(test_case)

    correctness_results.append({
        'index': index,
        'question': row['question'],
        'parsed_answer': row['parsed_answer'],
        'ground_truth': row['ground_truth'],
        'context': row['context'],
        'correctness_score': correctness_metric.score,
        'correctness_reason': correctness_metric.reason
    })

    answer_relevancy_results.append({
        'index': index,
        'question': row['question'],
        'parsed_answer': row['parsed_answer'],
        'ground_truth': row['ground_truth'],
        'context': row['context'],
        'answer_relevancy_score': answer_relevancy_metric.score,
        'answer_relevancy_reason': answer_relevancy_metric.reason
    })


correctness_df = pd.DataFrame(correctness_results)
answer_relevancy_df = pd.DataFrame(answer_relevancy_results)

# Save the results to a CSV file
correctness_df.to_csv('/content/drive/MyDrive/CS4371/GOT Results/GEval Results/Run 5/correctness_results.csv', index=False)
answer_relevancy_df.to_csv('/content/drive/MyDrive/CS4371/GOT Results/GEval Results/Run 5/answer_relevancy_results.csv', index=False)

Output()

Event loop is already running. Applying nest_asyncio patch to allow async execution...

Output()

Event loop is already running. Applying nest_asyncio patch to allow async execution...

Output()

...

Event loop is already running. Applying nest_asyncio patch to allow async execution...

Output()

Event loop is already running. Applying nest_asyncio patch to allow async execution...

Output()

Event loop is already running. Applying nest_asyncio patch to allow async execution...

In [ ]:
import seaborn as sb
import matplotlib.pyplot as plt

plt.figure(figsize=(24, 10))  # Width = 24 inches, Height = 10 inches
ax = plt.gca()

# Plot each metric
sb.lineplot(x=range(1, 54), y=score["correctness"], ax=ax, label='Answer Correctness')
sb.lineplot(x=range(1, 54), y=score["answer_relevancy"], ax=ax, label='Answer Relevancy',color='green')

# Customize the plot
ax.set_xticks(range(1, 54))
ax.set_xlabel('Questions')
ax.set_ylabel('Metrics: Answer Correctness, Context Recall, Answer Relevancy')
ax.set_title("GEval results using model llama2-7B [run1]")
ax.legend(title='Metrics')